<a href="https://colab.research.google.com/github/Gk58IISERTvm98/Text_Generation/blob/main/LSTM_Text_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
import numpy as np

In [2]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku
import numpy as np
import string

In [3]:
path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

600901/600901 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Corpus length: 600893


In [4]:
import textwrap
def wrap(x):
  return textwrap.fill(x, replace_whitespace=False, fix_sentence_endings=True)

In [ ]:
data = wrap(text)

In [6]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
def dataset_preparation(data):
  data = data.lower().split("\n")
  tokenizer.fit_on_texts(data)
  total_words = len(tokenizer.word_index) + 1

  # create input sequences using list of tokens
  input_sequences = []
  for line in data:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
      n_gram_sequence = token_list[:i+1]
      input_sequences.append(n_gram_sequence)

  # pad sequences
  max_sequence_len = max([len(x) for x in input_sequences])
  input_sequences = np.array(tf.keras.utils.pad_sequences(input_sequences,
                        maxlen=max_sequence_len, padding='pre'))

  # create predictors and label
  predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
  label = ku.to_categorical(label, num_classes=total_words)

  return predictors, label, max_sequence_len, total_words


In [7]:
predictors, label, max_sequence_len, total_words = dataset_preparation(data)

In [8]:
def create_model(max_sequence_len, total_words):
  input_len = max_sequence_len - 1
  model = Sequential()
  model.add(Embedding(total_words, 10, input_length=input_len))
  model.add(LSTM(150))
  model.add(Dropout(0.1))
  model.add(Dense(total_words, activation='softmax'))
  model.compile(loss='categorical_crossentropy', optimizer='adam')
  model.fit(predictors, label, epochs=10, verbose=1)
  print(model.summary())
  return model


In [9]:
model = create_model(max_sequence_len, total_words)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
2641/2641 ━━━━━━━━━━━━━━━━━━━━ 125s 46ms/step - loss: 6.7887
Epoch 2/10
2641/2641 ━━━━━━━━━━━━━━━━━━━━ 146s 48ms/step - loss: 6.2170
Epoch 3/10
2641/2641 ━━━━━━━━━━━━━━━━━━━━ 125s 47ms/step - loss: 5.9797
Epoch 4/10
2641/2641 ━━━━━━━━━━━━━━━━━━━━ 124s 47ms/step - loss: 5.7933
Epoch 5/10
2641/2641 ━━━━━━━━━━━━━━━━━━━━ 126s 48ms/step - loss: 5.6094
Epoch 6/10
2641/2641 ━━━━━━━━━━━━━━━━━━━━ 127s 48ms/step - loss: 5.4361
Epoch 7/10
2641/2641 ━━━━━━━━━━━━━━━━━━━━ 136s 46ms/step - loss: 5.2435
Epoch 8/10
2641/2641 ━━━━━━━━━━━━━━━━━━━━ 131s 42ms/step - loss: 5.0920
Epoch 9/10
2641/2641 ━━━━━━━━━━━━━━━━━━━━ 142s 42ms/step - loss: 4.9135
Epoch 10/10
2641/2641 ━━━━━━━━━━━━━━━━━━━━ 143s 42ms/step - loss: 4.7311


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 15, 10)              │         102,620 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 150)                 │          96,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 150)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 10262)               │       1,549,562 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 5,246,348 (20.01 MB)

 Trainable params: 1,748,782 (6.67 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,497,566 (13.34 MB)

None


In [10]:
model.fit(predictors, label, epochs=10, verbose=5)

Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10


In [ ]:
# Save the model in HDF5 format
model.save('my_model.h5')

In [12]:
def generate_text(seed_text, next_words, max_sequence_len, model):
  for j in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = tf.keras.utils.pad_sequences([token_list], maxlen=
                          max_sequence_len-1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted = np.argmax(predicted, axis=-1)

    output_word = ""
    for word, index in tokenizer.word_index.items():
      if index == predicted:
        output_word = word
        break
    seed_text += " " + output_word
  return seed_text

In [13]:
input_txt = "rely to giths of sayquitt guratuch with that this if"

In [14]:
print(generate_text(input_txt, 20, max_sequence_len, model))

rely to giths of sayquitt guratuch with that this if it is a matter of the same soul and the most sudden and of the reins the abysses of the
